#Material Genome 

The code below reads in the original "melting points.txt" document and extract all binary metal systems. This was required as a list of material systems was required to make the api calls to the server.

In [1]:
import os
import collections
import json
import requests

from itertools import izip
import warnings
warnings.filterwarnings('ignore')

# Read in the file and exxtract every entry in the dataset:
INPUT_FILE_NAME = "melting_points copy.txt"

input_data = open(INPUT_FILE_NAME)
element_systems = []
for line in input_data:
    line = line.strip().split(' ',1)
    if (len(line) > 1) and ("-" in line [0]):
        element_systems.append(line[0])
    else: 
        continue
input_data.close()

element_systems = set(element_systems)

The Below cell extracts all sub-directory names. this is required as a list of properties required is to be passed on to the API for each material. (Requires cleaning, this gives some unwanted properties also)

In [2]:
# Read names of all files in the sub-directory /mapidocs/materials (forked form github)

features = next(os.walk('./mapidoc/materials'))[1]

Funtion to flatten the resulting dict of dicts in the API call(stackoverflow : http://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys). 
Eg output: 

 `flatten({'a': 1, 'c': {'a': 2, 'b': {'x': 5, 'y' : 10}}, 'd': [1, 2, 3]})`

{'a': 1, 'c_a': 2, 'c_b_x': 5, 'd': [1, 2, 3], 'c_b_y': 10}

In [3]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

This Code should send repeated calls to the server and extract all information (Working). TODO: Convert output list of dictionaries into PANDAS dataframe, Clean data, Extra: Add loading bar

In [ ]:
mat_data = []
for e in element_systems:
    data = {
    'criteria': {"chemsys" : e},
    'properties': features
    }
    r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': '87kcBbIgsJmN3nTh'},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
    response_content = r.json() # a dict
    for item in response_content['response']:
        t = flatten(item)
        mat_data.append(t) 

## Examples for One system "Al-LU"

Example of one system and corresponding outpus below

In [27]:
data = {
    'criteria': {"chemsys" : "Al-Lu", "nelements" : 2},
    'properties': features
}
r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': 'S68xz1F7NhXAZWbI'},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
response_content = r.json() # a dict


In [28]:
len (response_content['response']) # need to figure out a way to extract information from this 

4

In [2]:
response_content['response'];

In [40]:
next(os.walk('./mapidoc/materials'))[1]

In [27]:
len(element_systems)

25297

This code snippet takes all task_ids from the material database. Could be useful to pull everything.

In [9]:
from pymatgen import MPRester
m = MPRester()
% env MAPI_KEY= "S68xz1F7NhXAZWbI"

materials_id = m.query(criteria={}, properties=["task_id"])

env: MAPI_KEY="S68xz1F7NhXAZWbI"


In [16]:
for a in materials_id:
    

[1]

In [ ]:
data = pd.DataFrame()
for item in response_content['response']:
    temp = flatten (item)
    temp = pd.DataFrame.from_dict(temp)
    data = pd.concat([data,temp])